In [3]:
!pip install scikit-learn


[notice] A new release of pip is available: 24.3.1 -> 25.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [6]:
from sklearn.metrics.pairwise import cosine_similarity
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings as wr
wr.filterwarnings('ignore')

In [7]:
customers=pd.read_csv("Customers.csv")
Products=pd.read_csv("Products.csv")
Transactions=pd.read_csv("Transactions.csv")

In [8]:
transaction_features = Transactions.groupby('CustomerID').agg({'TotalValue': 'sum'}).reset_index()

In [9]:
customer_feature = customers[['CustomerID', 'Region']]
customer_feature['Region'] = customer_feature['Region'].astype('category').cat.codes

In [10]:
mergedfeatures = pd.merge(customer_feature, transaction_features, on='CustomerID')
mergedfeatures['TotalValue'] = mergedfeatures['TotalValue'] / mergedfeatures['TotalValue'].max()

In [11]:
similaritymatrix = cosine_similarity(mergedfeatures[['Region', 'TotalValue']])

In [12]:
lookalikes = {}
for i, custid in enumerate(mergedfeatures['CustomerID'][:20]):
    similarindices = np.argsort(similaritymatrix[i])[-4:-1]
    lookalikes[custid] = [(mergedfeatures['CustomerID'][j], round(similaritymatrix[i][j], 4)) for j in similarindices]

In [13]:
lookalikedf = pd.DataFrame.from_dict(lookalikes, orient='index', columns=['1st Similar', '2nd Similar', '3rd Similar'])
lookalikedf.index.name = 'CustomerID'
lookalikedf.to_csv('Lookalike.csv')

print("Lookalike saved as 'Lookalike.csv'.")

Lookalike saved as 'Lookalike.csv'.
